In [28]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.ensemble import IsolationForest

In [4]:
df = pd.read_csv('Mall_Customers.csv')

In [5]:
Xtrain, Xtest = train_test_split(df, test_size=0.2, random_state=1)

In [32]:
num_cols = ['Annual Income (k$)', 'Age']


def delete_outliers(df):
    df_copy = df.copy()

    def calculate_iqr_boundaries(series):
        q25 = series.quantile(0.25)
        q75 = series.quantile(0.75)

        iqr = q75 - q25

        return (q25 - 1.5 * iqr, q75 + 1.5 * iqr)

    age_bounds = calculate_iqr_boundaries(df_copy['Age'])
    df_copy.loc[(df_copy['outlier_age'] == -1), 'Age'] = int(age_bounds[1])

    income_bounds = calculate_iqr_boundaries(df_copy['Annual Income (k$)'])
    df_copy.loc[(df_copy['outlier_income'] == -1), 'Annual Income (k$)'] = int(income_bounds[1])

    return df_copy

def find_outliers(df):
    df_copy = df.copy()

    isf = IsolationForest(n_estimators=100, contamination=0.005, random_state=1)
    
    y_pred = isf.fit_predict(Xtrain[['Age', 'Spending Score (1-100)']])
    df_copy['outlier_age'] = y_pred

    y_pred_inc = isf.fit_predict(Xtrain[['Annual Income (k$)', 'Spending Score (1-100)']])

    df_copy['outlier_income'] = y_pred_inc

    return df_copy


outliers = Pipeline(
        steps=[
            ('find_outliers', FunctionTransformer(find_outliers)),
            ('del_outliers', FunctionTransformer(delete_outliers)),
        ]
    )


imputers = Pipeline(
    steps=[
        ('num_imputer', SimpleImputer(strategy='median')),
        ('cat_imputer', SimpleImputer(strategy='most_frequent')),
    ]
)



preprocessor = ColumnTransformer(
        transformers=[
            ('imputers', imputers, num_cols),
            ('outliers', outliers),
            # ('categorical', categorical_transformer, make_column_selector(dtype_include='object')),
            # ('column_dropper', 'drop', columns_to_drop),
        ]
    )

x = preprocessor.fit_transform(Xtrain)
# y = preprocessor.transform(Xtrain)



ValueError: not enough values to unpack (expected 3, got 2)

In [26]:
x.get_feature_names_out()

array(['imputers__Annual Income (k$)', 'imputers__Age'], dtype=object)

In [ ]:
y

In [27]:
new = pd.DataFrame(y, columns=x.get_feature_names_out().tolist())
new

,imputers__Annual Income (k$),imputers__Age
0,63.0,68.0
1,63.0,54.0
2,103.0,36.0
3,20.0,37.0
4,44.0,51.0
...,...,...
155,72.0,31.0
156,73.0,32.0
157,50.0,60.0
158,75.0,57.0
